In [2]:
import pandas as pd 
import json 

In [ ]:
df = pd.read_json("/Users/ethantrang/Documents/zaloai_challenge_2023/raw/math_train.json")

def make_col(row):
    for key, value in row["data"].items():
        row[key] = value 
    return row 

df = df.apply(make_col, axis=1)
del df["__count__"]
del df["data"]
df = df[["id", "question", "choices", "explanation", "answer"]]
df.to_csv("math_train.csv",index=False)

In [3]:
df = pd.read_csv("/Users/ethantrang/Documents/zaloai_challenge_2023/data/math_train.csv")
df

,id,question,choices,explanation,answer
0,1,"Một người bán hàng bỏ ra 80,000 đồng tiền vốn ...","['A. 80,000 : 6', 'B. 80,000 x 6', 'C. 80,000 ...","Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồn...","D. (80,000 x 6) : 100"
1,10,8 dm2 24 cm2 = ……… dm2. Số thích hợp điền v...,"['A. 824', 'B. 82,4', 'C. 8,24', 'D. 0,824']","Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = ...","C. 8,24"
2,100,"10% của 11,5m2 là:","['A. 10,15dm2', 'B. 1,5m2', 'C. 15,5m2', 'D. 1...","10% của 11,5m2 là: 11,5 ${\times}$ 10 : 100 = ...","D. 1,15m2"
3,1004,"Một tam giác có đáy bằng 15 cm và gấp 2,5 l...","['A. 90 cm2', 'B. 37,5 cm2', 'C. 45 cm2', 'D. ...","Chiều cao của tam giác đó là: 15 : 2,5 = 6 (c...",C. 45 cm2
4,1005,Một tấm thiệp hình chữ nhật có chiều rộ...,"['A. 162 cm2', 'B. 81 cm2', 'C. 99 cm2', 'D. 6...",Chiều dài hình chữ nhật là: 9 ${\times}$ 2 ...,A. 162 cm2
...,...,...,...,...,...
1195,991,"Một ô tô đi được 60 km với vận tốc 60 km/giờ, ...","['A. 1,5 giờ', 'B. 2 giờ', 'C. 3 giờ', 'D. 4 g...","vì đoạn đường thứ nhất ô tô đi hết 1 giờ, đoạn...",C. 3 giờ
1196,993,"Một ô tô đi từ A đến B với vận tốc 38,5k...","['A. 2 giờ 1 phút', 'B. 2 giờ 6 phút', 'C. 2...","Thời gian ô tô đã đi là: 100,1 : 38,5 = 2,6 (...",C. 2 giờ 36 phút
1197,994,Một ô tô đi từ Hà Nội lúc 11 giờ trưa và đến V...,"['A. 4 giờ 30 phút', 'B. 6 giờ 30 phút', 'C. 6...",5 giờ 30 phút chiều = 17 giờ 30 phút\n Thời gi...,D. 6 giờ
1198,999,Một sân vận động hình chữ nhật có chiều ...,"['A.380 m', 'B.285 m', 'C.670 m', 'D.760 m']",Chiều dài sân vận động là: 95 ${\times}$ 3 ...,D.760 m


In [ ]:
df = pd.read_csv("/Users/ethantrang/Documents/zaloai_challenge_2023/data/math_train.csv")

def build_prompt(row):
    row["prompt"] = "Given the question:\n" + row["question"] + "\n" + "\n".join(eval(row["choices"])) + "\n" + "The answer is: " + str(row["explanation"]) + " Vì vậy đáp án là: " + row["answer"]
    return row 

df = df.apply(build_prompt, axis=1)
result = df["prompt"].to_list()
with open('prompts.json', 'w') as outfile:
    json.dump(result, outfile, ensure_ascii=False)

In [1]:
import pandas as pd
import torch
import json
from transformers import BloomTokenizerFast, BloomForCausalLM, Trainer
from transformers.training_args import TrainingArguments
from datasets import load_dataset

# Loading bloomz model and tokenizer 
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloomz-560m")

# Loading dataset prompts.json built using de portuguese legalQA dataset
dataset = load_dataset("json", data_files="/Users/ethantrang/Documents/zaloai_challenge_2023/data/prompts.json")

# # prepare the data for training
# def prepare_train_data(data):
#     # prompt + completion
#     text_input = data['text']
#     # tokenize the input (prompt + completion) text
#     tokenized_input = tokenizer(text_input, return_tensors='pt', padding=True)
#     # generative models: labels are the same as the input
#     tokenized_input['labels'] = tokenized_input['input_ids']
#     return tokenized_input

# train_dataset = dataset['train'].map(prepare_train_data, 
#                                      batched=True, 
#                                      remove_columns=["text"])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Failed to read file '/Users/ethantrang/Documents/zaloai_challenge_2023/data/prompts.json' with error <class 'AttributeError'>: 'str' object has no attribute 'keys'


DatasetGenerationError: An error occurred while generating the dataset

In [1]:
# pip install -q transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/bloomz-560m"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))


/Users/ethantrang/anaconda3/envs/zaloai/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translate to English: Je t’aime. I love you.</s>
